# ADP-Problem 

The goal of ths spreadsheed is too test methods on the Advection-Diffusion-production Problem from
DOI: 10.1002/pamm.201900209


#### Note on the Testproblem:

The production terms

$$u_1'= -\frac{u_1u_2}{0.01=u_1} + \cdots$$

$$u_2'= \frac{u_1u_2}{0.01=u_1} + \cdots$$

leed to unpredictable behavior when integrated with implicit methods.

This is caused by the stage equation ($y_i = u' + \Delta t a_{i,i} f(t,y_i)$) having a second root with negative values of $u_1$.
This can be fixed by replacing the function for negative $u_1 < -0.005$ with the value for $u_1 = -0.005$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg

from numba import jit, float64, stencil

fe =rk.loadRKM('FE').__num__()
rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

ck5 = rk.loadRKM('CK5').__num__()
dp5 = rk.loadRKM('DP5').__num__()
pd8 = rk.loadRKM('PD8').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()



#Extrapolation method
ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()
ex4 = rk.extrap(4,'implicit euler').__num__()
ex5 = rk.extrap(5,'implicit euler').__num__()
ex6 = rk.extrap(6,'implicit euler').__num__()
ex8 = rk.extrap(8,'implicit euler').__num__()

from OrderCondition import *
from RKimple import *
import utils 

## At first test the Productio Part



In [ ]:
@jit(float64[:](float64,float64[:]),nopython=True)
def f_prod(t,u):
        du = np.zeros(4)
        du[0] = 0.01*u[1]+0.01*u[2]+0.003*u[3]-(u[0]*u[1])/(0.01+u[0])
        du[1] = (u[0]*u[1])/(0.01+u[0])-0.01*u[1]-0.5*(1-np.exp(-1.21*u[1]**2))*u[2]-0.05*u[1]
        du[2] = 0.5*(1-np.exp(-1.21*u[1]**2))*u[2]-0.01*u[2]-0.02*u[2]
        du[3] = 0.05*u[1] + 0.02*u[2]-0.003*u[3]
        return du

@jit(float64[:](float64,float64[:]),nopython=True)
def f_prod_pos(t,u):
        du = np.zeros(4)
        u = np.where(u>-0.005,u,-0.005)

        du[0] = 0.01*u[1]+0.01*u[2]+0.003*u[3]-(u[0]*u[1])/(0.01+u[0])
        du[1] = (u[0]*u[1])/(0.01+u[0])-0.01*u[1]-0.5*(1-np.exp(-1.21*u[1]**2))*u[2]-0.05*u[1]
        du[2] = 0.5*(1-np.exp(-1.21*u[1]**2))*u[2]-0.01*u[2]-0.02*u[2]
        du[3] = 0.05*u[1] + 0.02*u[2]-0.003*u[3]
        return du



### Explicit method SSP104

In [ ]:
#u0 = np.array([1.,1.,1.,1.])

u0 = np.array([8.,2.,1.,4.])
dt = 0.035

t,u,b,KK = RK_variable_b(ssp104,dt,f_prod,u0,t_final=5,b_fixed=False,dumpK=True)

In [ ]:
plt.plot(t,u[0,:],label='1')
plt.plot(t,u[1,:],label='2')
plt.plot(t,u[2,:],label='3')
plt.plot(t,u[3,:],label='4')
plt.grid()
plt.legend()

In [ ]:
u[u<0]

In [ ]:
#plt.plot(t[1:],b.T[1:,:])
plt.plot(10*b.T)
plt.plot(u[0,:])
plt.grid()
plt.xlim([60,70])


In [ ]:
plt.plot(t[1:],np.linalg.norm((b[:,1:].T-ssp104.b),ord = 1,axis=1))
plt.grid()

#print(np.linalg.norm(b[:,1:].T-ssp104.b,ord = 1,axis=1).shape)

In [ ]:
#np.unravel_index(np.argmax(b),b.shape)
t[66]

for $\Delta t = 0.035$ at the time $t= 2.31$ a step occurs in $u_1$ and $u_2$. If we take a look at the $b$ of the corresponding step (index 66) it is clear that the regular method is used for the integration at this step. 
To futher investigate this behaiviour we dump the $K$ matricies.

In [ ]:
print(t[64])
print(b[:,64])
print(b[:,65])
print(b[:,66])

In [ ]:
print(u[:,65])
print(f_prod(0,u[:,65]))

In [ ]:
plt.plot(KK[10][0,:],'o',label='10')

plt.plot(KK[64][0,:],'o',label='64')
plt.plot(KK[65][0,:],'o',label='65')
plt.plot(KK[66][0,:],'o',label='66')
plt.plot(KK[67][0,:],'o',label='67')
plt.legend()

If we plot the values of the stages for $u_1$ we can see that for many steps all the values are relatively small. 
An exeption is the 5th stage. 
Here the values are $\ll 0$ for $n=64,65$ and $\gg$ for $n=66$ 
This apparently leads to the glitch.
Quesion: Why is the 5th stage so prone for outliers -> The stage evaluates the RHS at a u with negative values

The Errors are not caused by large values of b but by problems in the intermediate stages
Is the conclution that keeping $b_{new}-b_{old}$ small is necesary stil justified?


Conclusion: it si important to control for negative Values in the intermediate stages.
Possible solutions: Solve an positifity enshuring optimisation Problem at every intermediate step.
Can be formulated as chosing a new $b$ for every intermediate stage which then gets part of $A$ in the next step.

Another idea to explore (Is kind of obvoious but could also be stupid): Replace $f(t,u)$ with an $f_{pos}(t,u)$ with 
$$f(t,u)-f_{pos}(t,u) = 0 \forall u \geq 0 $$
Whereas $f_{pos}(t,u)$ is welldefined for $u \leq 0$

We can see that the glitch still happens

### Explicit Method CK5/DP5

In [ ]:
print(dp5)
print(dp5.p)
print(ck5)
print(ck5.p)

In [ ]:
#u0 = np.array([1.,1.,1.,1.])

u0 = np.array([8.,2.,1.,4.])
dt = 0.006

t,u,b,KK = RK_variable_b(dp5,dt,f_prod,u0,t_final=10,b_fixed=False,dumpK=True,solver='scipy_ip')

In [ ]:
#plt.plot(t,u[0,:],label='1')
#plt.plot(t,u[1,:],label='2')
#plt.plot(t,u[2,:],label='3')
#plt.plot(t,u[3,:],label='4')
plt.plot(u[0,:],label='1')
plt.plot(u[1,:],label='2')
plt.plot(u[2,:],label='3')
plt.plot(u[3,:],label='4')
plt.grid()
plt.legend()

In [ ]:
plt.plot(t[1:],b.T[1:,:])
plt.grid()

## See for timesteps

In [ ]:
u0 = np.array([8.,2.,1.,4.])

methods = (ssp104,dp5,ck5,ssp2,ssp3,merson4,pd8)
dt = utils.findall_dt(RK_variable_b,methods,f_prod,u0,dt_start=0.1,tol=0.001,
                   Params=dict(t_final=50,b_fixed=True,fallback = True,num_fallback= 1,
                      LP_opts = {'reduce':True,'feastol':1e-10}))


In [ ]:
utils.plot_times(methods,dt,effective=True,title='ADP_problem')
plt.ylim(0,0.05/10)

In this plot the bars for $\Delta t_{feasible}$ are cropped because they are extremely large because for the low dimentional case there are many solutions for the LP-Problem. 
It can be seen that the SSP104 method does only ensure positifity for a part of the stable $\Delta t$. 

### Implicit method

In [ ]:
u0 = np.array([8.,2.,1.,4.])
dt = 1

t,u,b = RK_variable_b_implicit(ex4,dt,f_prod_pos,u0,t_final=10,b_fixed=False,solver_eqs =solver_nonlinear_arg,
                               fallback=True,num_fallback=1)



In [ ]:
plt.plot(t,u[0,:],label='1')
plt.plot(t,u[1,:],label='2')
plt.plot(t,u[2,:],label='3')
plt.plot(t,u[3,:],label='4')
plt.grid()
plt.legend()

In [ ]:
plt.plot(t[1:],b.T[1:,:])
#plt.plot(b.T[1:,:])
plt.grid()
#plt.xlim([60,70])

In [ ]:
plt.plot(t[1:],ex4.b-b.T[1:,:])

# PDE

In [ ]:
# Now we implement the advection Diffusion equation
# To arcieve compatibility with the existing time integrators the u's are stored in a single vector

#u= [u_1^T,u_2^T,u_3^T,u_4^T]^T




@stencil
def kernel_adv(a,dxi):
    return dxi*(a[0]-a[-1])

@stencil
def kernel_dif(a,dxqi):
    return dxqi*(a[-1]-2*a[0]+a[1])

@jit(float64[:](float64[:],float64),nopython=True)
def adv(u,dxi):
    du = kernel_adv(u,dxi)
    
    #Periodic
    du[0]=dxi*(u[0]-u[-1])
    return du

@jit(float64[:](float64[:],float64),nopython=True)
def dif(u,dxqi):
    du = kernel_dif(u,dxqi)

    #Periodic
    du[0] =dxqi*(u[-1]-2*u[0]+u[1])
    du[-1]=dxqi*(u[-2]-2*u[-1]+u[0])
    
    return du


@jit(float64[:](float64,float64[:]),nopython=True)
def f_ADP(t,u):
    a=1e-2
    d=1e-6
    
    du = np.zeros_like(u)
    
    #split u in different parts
    N = len(u)//4
    u1 = u[0:N]
    u2 = u[N:2*N]
    u3 = u[2*N:3*N]
    u4 = u[3*N:4*N]
    
    #calculate Production
    pu1 = 0.01*u2+0.01*u3+0.003*u4-(u1*u2)/(0.01+u1)
    pu2 = -0.01*u2-0.5*(1-np.exp(-1.21*u2**2))*u3-0.05*u2+(u1*u2)/(0.01+u1)
    pu3 = 0.5*(1-np.exp(-1.21*u2**2))*u3-0.01*u3-0.02*u3
    pu4 = 0.05*u2 + 0.02*u3-0.003*u4
    
    #Advection and Diffusion
    dx=1/N #periodic boundary
    dxq = dx**2
    
    
    du[0:N] = -a*adv(u1,1/dx)+d*dif(u1,1/dxq) + pu1 
    du[N:2*N] = -a*adv(u2,1/dx)+d*dif(u2,1/dxq) + pu2
    du[2*N:3*N] = -a*adv(u3,1/dx)+d*dif(u3,1/dxq) + pu3
    du[3*N:4*N] = -a*adv(u4,1/dx)+d*dif(u4,1/dxq) + pu4   
    
    return du


#Added a modification to the function that alters the behavior for values <0.005 to make shure that
#there is no second root of the stageeqation with negative us
@jit(float64[:](float64,float64[:]),nopython=True)
def f_ADP_pos(t,u):
    a=1e-2
    d=1e-6
    
    du = np.zeros_like(u)
    
    #split u in different parts
    N = len(u)//4
    u1 = np.where(u[0:N]>-0.005,u[0:N],-0.005)   #
    u2 = np.where(u[N:2*N]>-0.005,u[N:2*N],-0.005)
    u3 = np.where(u[2*N:3*N]>-0.005,u[2*N:3*N],-0.005)
    u4 = np.where(u[3*N:4*N]>-0.005,u[3*N:4*N],-0.005)
    
    
    
    #calculate Production
    pu1 = 0.01*u2+0.01*u3+0.003*u4-(u1*u2)/(0.01+u1)
    pu2 = -0.01*u2-0.5*(1-np.exp(-1.21*u2**2))*u3-0.05*u2+(u1*u2)/(0.01+u1)
    pu3 = 0.5*(1-np.exp(-1.21*u2**2))*u3-0.01*u3-0.02*u3
    pu4 = 0.05*u2 + 0.02*u3-0.003*u4
    
    #Advection and Diffusion
    dx=1/N #periodic boundary
    dxq = dx**2
    
    
    du[0:N] = -a*adv(u1,1/dx)+d*dif(u1,1/dxq) + pu1 
    du[N:2*N] = -a*adv(u2,1/dx)+d*dif(u2,1/dxq) + pu2
    du[2*N:3*N] = -a*adv(u3,1/dx)+d*dif(u3,1/dxq) + pu3
    du[3*N:4*N] = -a*adv(u4,1/dx)+d*dif(u4,1/dxq) + pu4   
    
    return du


def precond_ADP(t,u,dt,a,f): #solves the impicit Advection and diffusion Problem without the Production part
    a=1e-2
    d=1e-6
    
    y0 = np.zeros_like(u)
    
    N = len(u)//4
    u1 = u[0:N]
    u2 = u[N:2*N]
    u3 = u[2*N:3*N]
    u4 = u[3*N:4*N]
    
    #Advection and Diffusion
    dx=1/N #periodic boundary
    dxq = dx**2
    
    A_dif = 1/dx**2 * (-2*np.diag(np.ones(N))+np.diag(np.ones(N-1),-1)+np.diag(np.ones(N-1),1))
    A_dif[-1,0] = 1/dx**2
    A_dif[0,-1] = 1/dx**2

    A_adv = 1/dx * (np.diag(np.ones(N))-np.diag(np.ones(N-1),-1))
    A_adv[0,-1] = 1/dx
    
    A = -a*A_adv + d*A_dif
    
    y0[0:N] = u1+dt*a*solver_Matrix(t,u1,dt,a,A)
    y0[N:2*N] = u2+dt*a*solver_Matrix(t,u2,dt,a,A)
    y0[2*N:3*N] = u3+dt*a*solver_Matrix(t,u3,dt,a,A)
    y0[3*N:4*N] = u4+dt*a*solver_Matrix(t,u4,dt,a,A)  
    
    return y0
    
def precond_ADP_forward(t,u,dt,a,f):
    a=1e-2
    d=1e-6
    
    y0 = np.zeros_like(u)
    
    N = len(u)//4
    u1 = u[0:N]
    u2 = u[N:2*N]
    u3 = u[2*N:3*N]
    u4 = u[3*N:4*N]
    
    #Advection and Diffusion
    dx=1/N #periodic boundary
    dxq = dx**2
    
    A_dif = 1/dx**2 * (-2*np.diag(np.ones(N))+np.diag(np.ones(N-1),-1)+np.diag(np.ones(N-1),1))
    A_dif[-1,0] = 1/dx**2
    A_dif[0,-1] = 1/dx**2

    A_adv = 1/dx * (np.diag(np.ones(N))-np.diag(np.ones(N-1),-1))
    A_adv[0,-1] = 1/dx
    
    A = -a*A_adv + d*A_dif
    
    y0[0:N] = u1+dt*a*A@u1
    y0[N:2*N] = u2+dt*a*A@u2
    y0[2*N:3*N] = u3+dt*a*A@u3
    y0[3*N:4*N] = u4+dt*a*A@u4  
    
    return y0

In [ ]:
#Additional functions for better handling of System

def initial_ADP(N):
    start = np.array([8.,2.,1.,4.])
    u1s = np.zeros(N)
    u1s[0:N//2]=7
    u1s[N//2:]=8

    u2s = np.zeros(N)
    u2s[N//2:-1]=2

    u3s = np.zeros(N)
    u3s[N//2:]=1

    u4s = np.zeros(N)
    u4s[0:N//2]=1
    u4s[N//2:]=4


    u0=np.ones(N*4)
    u0[0:N] =  u1s
    u0[N:2*N] =  u2s
    u0[2*N:3*N] =  u3s
    u0[3*N:4*N] =  u4s
    
    return u0
    

def plot_image(u,t):
    N = u.shape[0]//4
    cmap=plt.get_cmap('plasma')
    display(u.shape)
    extent=[0,t[-1],1,0]
    plt.subplot(4, 1, 1)
    plt.imshow(u[0:N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')
    plt.subplot(4, 1, 2)
    plt.imshow(u[N:2*N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')
    plt.subplot(4, 1, 3)
    plt.imshow(u[2*N:3*N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')
    plt.subplot(4, 1, 4)
    plt.imshow(u[3*N:4*N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')

def plot_at_time(u,T):
    n=np.argmin(np.abs(t-T))

    print(n,t[n])


    u_ = u[:,n]
    N= len(u_)//4

    u1 = u_[0:N]
    u2 = u_[N:2*N]
    u3 = u_[2*N:3*N]
    u4 = u_[3*N:4*N]

    plt.plot(u1,label='u1')
    plt.plot(u2,label='u2')
    plt.plot(u3,label='u3')
    plt.plot(u4,label='u4')
    
    plt.title('T='+str(T))
    plt.legend()

### Explicit

At first we want to investigate the behavior of the explicit mehthods. 

Questions:

$\bullet$ which method is usefull, we would expect that the dp5 method is more adventagios for our case because it should cause problems with positifity wth smaler timesteps.

$\bullet$ Do we get poblems with infeasible Problems. 

$\bullet$ Can we use high orders for solvers with $s > p$ or does the high order cause Problems


Note: For many testcases the LP solver crashed for unreasonable small timesteps. These issues could be solved by reducing the positifity conditons to the necessary conditions.


We are going to test the solvers for multiple timesteps. For the explicit methos we want to show the important times.

$$ \Delta t \; \underset{0}{+}-------------\underset{\Delta t_{pos}}{+}\underbrace{------------}_{\gamma}\underset{\Delta t_{feasible}}{+}----\underset{\Delta t_{stable}}{+}---->  $$

$\Delta t_{pos}$ is the largest stepsize that yields to positive results without adapting the $b$

$\Delta t_{feasible}$ is the largest stepsize that can be used with the adaptive $b$

$\Delta t_{stable}$ is the largest stepsize for that the method is stable


The timesteps where the adaption of the b is usefull is denoted with $\gamma$ idealy this interval is as large as possible. This means that $\Delta t_{pos} \ll \Delta t_{feasible} \approx \Delta t_{stable}$

With stepsizes $\Delta t > \Delta t_{stable} $ we cann not expect the solution to be reasonable

We will approximate the borders for the $ssp104$ method and the $dp5$ method

In [ ]:
N =100
x = np.linspace(0,1,num = N)
dx=1/(N) # no -1 because of periodic boundary
u0 = initial_ADP(100)

methods = (ssp104,dp5,ck5,ssp2,ssp3,merson4,pd8)
dt = utils.findall_dt(RK_variable_b,methods,f_ADP,u0,dt_start=0.009,tol=0.0001,
                   Params=dict(t_final=10,b_fixed=True,fallback = True,num_fallback= 1,
                      LP_opts = {'reduce':True,'feastol':1e-10}))
display(dt)   


In [ ]:
utils.plot_times(methods,dt,effective=False,title='ADP_problem')

We can see the same behavior of the ssp104 method as in the reaction Problem. This explains why the positifity is also only enured for a subset of stable timesteps. 
The dp5 and ck5 methods can handle the reaction part much better (compare to the plot for the Production test). This explains why $\Delta t_{stable} = \Delta t_{pos}$ for these both methods.
The proportions ot the plot do not change when speeding up and down the reaction part compared to the advection and diffusion part. Apparently the size of the timesteps is determined by the rection part.

For the ssp104 method the the region with positive results could be expanded to the hole region with stable timesteps.

In [ ]:
#dp5
N =100
x = np.linspace(0,1,num = N)
dx=1/(N) # no -1 because of periodic boundary



u0 = initial_ADP(N)




#dp5
#dt = 0.0031 #dt_pos

  #dt_feasible for fallback = False #imeadetly crashes

  #dt_feasible for num_fallback = 1 #imeadetly crashes

#dt = 0.0041 #dt_feasible for num_fallback = 2

#dt = 0.006 # stil feasible but shows strong distortions near start num_fallback= 3
dt = 0.0061 #dt_feasible for num_fallback = 3

#dt = 0.0031 # dt_stable

dt = 0.008
dt = 0.015


t,u,b,status = RK_variable_b(ssp104,dt,f_ADP,u0,t_final=50,b_fixed=False,fallback = True,num_fallback= 3,
                      LP_opts = {'verbose_LP':False,'reduce':True,'feastol':1e-10},return_status=True,verbose=False)

#t,u,b = RK_variable_b(dp5,dt,f_ADP,u0,t_final=50,b_fixed=False,fallback = True,num_fallback= 2,solver='scipy_ip',
#                      LP_opts = {'reduce':True})


display(status)

In [ ]:
#plot_image(((u<0)*1.)[:,range(1,u.shape[1],10)],t)
plot_image(u[:,range(1,u.shape[1],10)],t)

In [ ]:
T= 22#8.82
plot_at_time(u,T)

In [ ]:
plt.plot(t[1:],b.T[1:,:]);

In [ ]:
print('min:',np.min(u))
print(u[u<0])
print(np.sum(u<0))

### Implicit

For implicit methods $\Delta t$ could be indefinitley large. This is not true in pracitice because at sime time the solver for the stage equation fails. Also does the solution no longer resemble the actual solution. 
There were some effords tried to increase the performance of the solver for the stage equations. But apparently the default scipy.fsolve works best.

In [ ]:
N =100
x = np.linspace(0,1,num = N)
dx=1/(N) # no -1 because of periodic boundary


u0 = initial_ADP(N)

#dt = 0.5 #solver ECOS fails for 0.4



dt = 0.7   #with dt = 0.7 solution does not resemble exact solution
t,u,b = RK_variable_b_implicit(ex3,dt,f_ADP_pos,u0,t_final=50,b_fixed=True,solver_eqs =solver_nonlinear_arg,
                               fallback=False,num_fallback=1,solver=cp.ECOS,LP_opts = {'reduce':True})
#fallback=True,num_fallback=1

#for dt = 0.5 no convergenc error with nk


#Fails, beginning with LP-solver note getting a better solution, in some parts reasonabl negative results
#t,u,b = RK_variable_b_implicit(ex3,dt,f_ADP_pos,u0,t_final=50,b_fixed=False,solver_eqs =solver_nonlinear_arg,
#                               fallback=False,num_fallback=1,solver='scipy_sim',LP_opts = {'reduce':True,'tol':1e-8})

In [ ]:
plot_image(u,t)


In [ ]:
plt.plot(t[1:],b.T[1:,:])
plt.grid()
#plt.xlim([15,20])

In [ ]:
u[u<0]

In [ ]:
T= 25#8.82
plot_at_time(u,T)

# Appendix Convergence 

In [ ]:
# calculate a reference solution is possibly still not extremle exact. Maybee using something with stepsize controll
u0 = initial_ADP(100)
N =100

t_ref,u_ref,b_ref = RK_variable_b(ssp104,0.00005,f_ADP,u0,t_final=9.,b_fixed=True) #dt old = 0.0001 #0.00005

In [ ]:
save = False
load = True
if save:
    np.savez('reference_ADP.npz', u_ref=u_ref, t_ref = t_ref, label=('ssp104','dt = 0.00001'))

if load:
    npzfile = np.load('reference_ADP.npz')
    t_ref = npzfile['t_ref']
    u_ref = npzfile['u_ref']

In [ ]:


#dt = np.logspace(-3,-0.5,num=3)
dt = np.array([0.05,0.1,0.3,0.4,0.5])

u0 = initial_ADP(100)

referencea = np.zeros([len(u0),len(dt)])
for i in range(len(dt)):
    referencea[:,i]= u_ref[:,-1]



N =100
x = np.linspace(0,1,num = N)
dx=1/(N) # no -1 because of periodic boundary

print('BE')
sol_a1,err_a1 = utils.plot_convergence(RK_variable_b_implicit,be,f_ADP_pos,u0,dt,referencea,error='rel',step = -1,
                              Params=dict(t_final=9.,b_fixed=True,solver_eqs =solver_nonlinear_arg))

print('ex3')
sol_a2,err_a2 = utils.plot_convergence(RK_variable_b_implicit,ex3,f_ADP_pos,u0,dt,referencea,error='rel',step = -1,
                              Params=dict(t_final=9.,b_fixed=True,solver_eqs =solver_nonlinear_arg))

print('ex3_adapted')
sol_a2,err_a2 = utils.plot_convergence(RK_variable_b_implicit,ex3,f_ADP_pos,u0,dt,referencea,error='rel',step = -1,
                              Params=dict(t_final=9.,b_fixed=False,solver_eqs =solver_nonlinear_arg,
                               fallback=True,num_fallback=1,solver=cp.ECOS,LP_opts = {'reduce':True}))



In [ ]:
np.argmin?

In [ ]:
np.linalg.norm?